In [1]:
import ee
import geemap
import numpy as np
import time
# import geopandas as gpd

In [2]:
ee.Initialize()

In [3]:
#shp path
tp_crop ='crop_pt.shp'
tp_non_crop = 'non_crop_pt.shp'
#TODO training data explaination 
#ee object
crop_ee = geemap.shp_to_ee(tp_crop)
noncrop_ee = geemap.shp_to_ee(tp_non_crop)
#feature collection
fc = crop_ee.merge(noncrop_ee)
#define bounding box
bbox = ee.Geometry.bounds(fc)
aoi = bbox
#TODO #read about system.index

In [4]:
#Sentinel 2 Image collection, cloudmask and mosaic
sdate= '2020-05-01'
edate= '2020-08-31'

def cloud_prob():
    innerJoined = ee.Join.inner().apply(**{
        'primary':ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterBounds(aoi)\
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10)),
         'secondary': ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")\
            .filterBounds(aoi),       
        'condition':ee.Filter.equals(**{
            'leftField':'system:index',
            'rightField':'system:index',
        })
    })
    def mergeBands(joinresult):
        return ee.Image(joinresult.get('primary'))\
            .addBands(joinresult.get('secondary'))
    newCollection = innerJoined.map(mergeBands)
    return ee.ImageCollection(newCollection)

collection = cloud_prob().filterDate(sdate, edate)

def maskcloud(_image):
    cloudProbabilityThreshold =10
    cloudMask = _image.select('probability').lt(cloudProbabilityThreshold)
    return _image.updateMask(cloudMask)

image = collection.map(maskcloud).median().clip(aoi)

In [5]:
#Resolution merge of 20m bands to 10m -TODO explain the process
panchro = (image.select('B2').add(image.select('B3'))\
               .add(image.select('B4')).add(image.select('B8')))\
               .divide(4).rename('pan')
image = image.addBands(panchro)

bands = ['B5','B6','B7','B8A','B9','B11','B12']

def sharpen(image):
    kernel = ee.Kernel.square(**{
        'radius': 20,
        'units':'meters'
    })
    toSharpen = image.select(bands)
    pan = image.select('pan')
    meanPan = pan.reduceNeighborhood(**{
        'reducer' : ee.Reducer.mean(),
        'kernel' : kernel
    })
    offset = pan.subtract(meanPan)
    return toSharpen.add(offset)
sharpened = sharpen(image)    

In [6]:
#image c0llection relevant bands (all in 10m), converted to float
bands10m = image.select(['B2','B3','B4','B8'])
image_feature = bands10m.addBands(sharpened).divide(10000)

In [7]:
#vegetation indices - TODO comment on the use of each VI
def Veg_Index(image_name):
    #NDVI
    ndvi = image_name.expression(
        '(NIR+RED)/(NIR - RED)', {
            'NIR': image_name.select('B8'),
            'RED': image_name.select('B4')
        }).rename('NDVI')
    #EVI
    evi = image_name.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',{
        'NIR': image_name.select('B8'),
        'RED': image_name.select('B4'),
        'BLUE': image_name.select('B2')
    }).rename('EVI')
    #GCVI Green Chlorophyll Vegetation Index - TODO read about GCVI use cases
    gcvi = image_name.expression(
    '(NIR / GREEN) - 1',{
        'NIR':image_name.select('B8'),
        'GREEN':image_name.select('B3')
    }).rename('GCVI')
    #RDGCVI
    gcvi = image_name.expression(
    '(NIR / REDE1) - 1',{
        'NIR':image_name.select('B8'),
        'REDE1':image_name.select('B5')
    }).rename('RDGCVI')
    
    return image_name.addBands(ndvi).addBands(evi).addBands(gcvi)
#image composite of all spectral indices and bands
image_featureWithVI = Veg_Index(image_feature)

In [7]:
#sentinel 1
sdate= '2020-05-01'
edate= '2020-08-31'
VVcollection = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filterBounds(aoi)\
    .filterDate(sdate,edate)\
    .select(['VV'])


In [8]:
Map = geemap.Map(center=(0.9613,34.6536),zoom=8)
Map

Map(center=[0.9613, 34.6536], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [9]:
Map.addLayer(image, {'bands': ['B9', 'B11', 'B12'], 'min': 0, 'max': 3000}, 'Original')
Map.addLayer(sharpened, {'bands': ['B9', 'B11', 'B12'], 'min': 0, 'max': 3000}, 'Sharpen 1')

In [12]:
#exprot to assest
task = ee.batch.Export.image.toAsset(**{'image':image_feature,
'description': 'Compositeband',
'assetId':'users/aadebowaledaniel/cpclasscomposite', 
'scale': 10, 
'region':aoi.getInfo()['coordinates'],
'maxPixels': 1e13,
})
task.start()
#Monitor the task
while task.status()['state'] in ['READY', 'RUNNING']:
  print(task.status())
  time.sleep(5)
else:
  print(task.status())

EGSYW27Y3SUI'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620784938141, 'update_timestamp_ms': 1620785399357, 'start_timestamp_ms': 1620784958958, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'FP4VZGG3MHGFEGSYW27Y3SUI', 'name': 'projects/earthengine-legacy/operations/FP4VZGG3MHGFEGSYW27Y3SUI'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620784938141, 'update_timestamp_ms': 1620785459352, 'start_timestamp_ms': 1620784958958, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'FP4VZGG3MHGFEGSYW27Y3SUI', 'name': 'projects/earthengine-legacy/operations/FP4VZGG3MHGFEGSYW27Y3SUI'}
{'state': 'RUNNING', 'description': 'Compositeband', 'creation_timestamp_ms': 1620784938141, 'update_timestamp_ms': 1620785459352, 'start_timestamp_ms': 1620784958958, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'FP4VZGG3MHGFEGSYW27Y3SUI', 'name': 'projects/earthengine-legacy/operations/FP4VZGG3MHGFEGSYW27Y3SUI'}
{'state': 'RUNNING', 'des

In [4]:
#import composite from asset
predimage = ee.Image('users/aadebowaledaniel/cbinary')

In [5]:
predbands = predimage.bandNames()

In [6]:
#continue from here
trainingSample  = predimage.sampleRegions(
    collection = fc,
    properties=['label'],
    scale = 10, 
    geometries = True
)
# trainingSample.size().getInfo()

rantraning = trainingSample.randomColumn('random')
split = 0.7
trainsample = rantraning.filter(ee.Filter.lt('random', split))
testsample = rantraning.filter(ee.Filter.gte('random', split))

#train random forest classifier
classifier = ee.Classifier.smileRandomForest(100).train(trainsample, 'label', predbands)


#confusion matrix train Accurracy
trainAccuracy = classifier.confusionMatrix()
print('Resubstitution error matrix: ', trainAccuracy.getInfo())
print('Training overall accuracy: ', trainAccuracy.accuracy().getInfo())

#classify features
testclassifier = testsample.classify(classifier)
#cconfusion matrix representing expected accuracy
testAccuracy = testclassifier.errorMatrix('label', 'classification')
print('confusion Matrix', testAccuracy.getInfo())
print('Valdation overall accuracy:', testAccuracy.accuracy().getInfo())

#classify image
Imageclassified = predimage.classify(classifier)
# altImageclassified = predimage.classify(classifier)

Resubstitution error matrix:  [[463, 8], [0, 1790]]
Training overall accuracy:  0.9964617425917736
confusion Matrix [[136, 60], [25, 776]]
Valdation overall accuracy: 0.9147442326980942


In [21]:
# Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 0.3, 'gamma': 2}, 'True colour composite')
Map.addLayer(s2_sr_cld_col_eval, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 1500, 'gamma': 2}, 'no mask')


In [11]:
cropmask = Imageclassified.eq(1)
# vis = {'pallete':['#18d848']}
Map.addLayer(cropmask,{'color':'green'}, 'Vegetation')

cropmask = Imageclassified.eq(2)

cropmaskdd = cropmask.updateMask(cropmask)

cropmask_vec = cropmaskdd.reduceToVectors(
    geometryType= 'polygon',
    reducer= ee.Reducer.countEvery(),
    scale= 10,
    maxPixels= 1e13,
    geometry= aoi
)
Map.addLayer(cropmask_vec, {'color':'green'}, 'Crop Mask')

In [15]:
# #convert to Vector
# cropmask = Imageclassified.eq(1)

# vector_cropmask = cropmask.updateMask(cropmask).reduceToVectors(
#     reducer=ee.Reducer.countEvery(),
#     geometry= aoi,
#     scale=10,
#     maxPixels=1e13)
# def sim(nna):
#     return nna.simplify(maxError = 100)
# vector_cropmask = vector_cropmask.map(sim)
# cropmaskVec = cropmask.reduceToVectors({
#     # 'geometryType': 'polygon',
#   'reducer': ee.Reducer.countEvery(),
#   'scale': 10,
#   'geometry':aoi,
# })

# vector_img = Imageclassified.unmask(0).reduceToVectors(
#         geometry = aoi,
#         scale = 10,
#         maxPixels = 1e13,
#         bestEffort =True
#     )
# taskvec = ee.batch.Export.table.toAsset(collection=vector_cropmask,
# description= 'Vegetation mask', assetId= 'users/aadebowaledaniel/vegetationmask')
# taskvec.start()
# #Monitor the task
# while taskvec.status()['state'] in ['READY', 'RUNNING']:
#   print(taskvec.status())
#   time.sleep(5)
# else:
#   print(taskvec.status())